To make it easier for users to use, Gentopia has expanded on the yaml format and added some features, such as referencing files and using environment variables.

The detailed list is shown in the following table:

| Feature | Syntax | Description                                                                                                                                                                                                                                                |
| --- | --- |------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Reference file | `!include` | This syntax can be used to reference other files, such as `!include ./config.yaml`. All contents of the file will be treated as yaml for parsing. This can be used to reference other agents.                                                              |
| Reference Prompt | `!prompt` | This syntax can be used to reference prompts, such as `!prompt ZeroShotVanillaPrompt`. This will reference a pre-defined prompt in Gentopia. To import a custom prompt, use the full Python path, such as `!prompt gentopia.prompt.ZeroShotVanillaPrompt`. |
| Use environment variable | `!env` | This syntax can be used to reference environment variables, such as `!env GENTOPIA_HOME`.                                                                                                                                                                  |
| Reference file | `!file` | This syntax can be used to reference files, such as `!file ./config.yaml`. All contents of the file will be treated as a string for parsing.                                                                                                               |

In [1]:
from gentopia.assembler.agent_assembler import AgentAssembler
from gentopia.output import enable_log
from gentopia import chat

This is a sample agent that uses the `!prompt` syntax to reference a pre-defined prompt in Gentopia.

In [4]:
assembler = AgentAssembler(file='configs/sample_agent.yaml')
agent = assembler.get_agent()
print(agent)

RewooAgent(
    name='mathria',
    type=<AgentType.rewoo: 'rewoo'>,
    version='0.0.1',
    description='A math agent capable of solving very complicated math problems. Ask it to solve very hard math 
problems.',
    target_tasks=['solving math problems', 'answer things about math'],
    llm={
        'Planner': OpenAIGPTClient(
            model_name='gpt-3.5-turbo-0613',
            params=OpenAIParamModel(
                max_tokens=1024,
                temperature=0.0,
                top_p=0.9,
                presence_penalty=0.0,
                frequency_penalty=0.0,
                n=1,
                stop=[]
            )
        ),
        'Solver': OpenAIGPTClient(
            model_name='gpt-3.5-turbo-0613',
            params=OpenAIParamModel(
                max_tokens=1024,
                temperature=0.0,
                top_p=0.9,
                presence_penalty=0.0,
                frequency_penalty=0.0,
                n=1,
                stop=[]
            )
        )
    },
    prompt_template={
        'Planner': PromptTemplate(
            input_variables=['tool_description', 'task'],
            output_parser=None,
            partial_variables={},
            template="You are an AI agent who makes step-by-step plans to solve a problem under the help of 
external tools. \nFor each step, make one plan followed by one tool-call, which will be executed later to retrieve 
evidence for that step.\nYou should store each evidence into a distinct variable #E1, #E2, #E3 ... that can be 
referred to in later tool-call inputs.    \n\n##Available Tools##\n{tool_description}\n\n##Output Format (Replace 
'<...>')##\n#Plan1: <describe your plan here>\n#E1: <toolname>[<input here>] (eg. Search[What is Python])\n#Plan2: 
<describe next plan>\n#E2: <toolname>[<input here, you can use #E1 to represent its expected output>]\nAnd so 
on...\n  \n##Your Task##\n{task}\n\n##Now Begin##\n",
            template_format='f-string',
            validate_template=True
        ),
        'Solver': PromptTemplate(
            input_variables=['plan_evidence', 'task'],
            output_parser=None,
            partial_variables={},
            template='You are an AI agent who solves a problem with my assistance. I will provide step-by-step 
plans(#Plan) and evidences(#E) that could be helpful.\nYour task is to briefly summarize each step, then make a 
short final conclusion for your task.\n\n##My Plans and Evidences##\n{plan_evidence}\n\n##Example Output##\nFirst, 
I <did something> , and I think <...>; Second, I <...>, and I think <...>; ....\nSo, <your conclusion>.\n\n##Your 
Task##\n{task}\n\n##Now Begin##\n',
            template_format='f-string',
            validate_template=True
        )
    },
    plugins=[
        WolframAlpha(
            name='wolfram_alpha',
            description='A WolframAlpha search engine. Useful when you need to solve a complicated Mathematical or 
Algebraic equation. Input should be an equation or function.',
            args_schema=<class 'pydantic.main.WolframAlphaArgs'>,
            verbose=False,
            handle_tool_error=False
        )
    ],
    args_schema=<class 'pydantic.main.ArgsSchema'>,
    memory=None,
    examples={}
)

This is a sample agent that uses the `!include` syntax to reference another agent.

In [5]:
assembler = AgentAssembler(file='configs/include.yaml')
agent = assembler.get_agent()
print(agent)

OpenAIFunctionChatAgent(
    name='main',
    type=<AgentType.openai: 'openai'>,
    version='0.0.1',
    description='main agent leveraging OpenAI function call API.',
    target_tasks=['anything'],
    llm=OpenAIGPTClient(
        model_name='gpt-4-0613',
        params=OpenAIParamModel(
            max_tokens=1024,
            temperature=0.0,
            top_p=0.9,
            presence_penalty=0.0,
            frequency_penalty=0.0,
            n=1,
            stop=[]
        )
    ),
    prompt_template=PromptTemplate(
        input_variables=['instruction'],
        output_parser=None,
        partial_variables={},
        template='{instruction}',
        template_format='f-string',
        validate_template=True
    ),
    plugins=[
        GoogleSearch(
            name='GoogleSearch',
            description='Tool that searches results from Google. Input should be a search query.',
            args_schema=<class 'pydantic.main.GoogleSearchArgs'>,
            verbose=False,
            handle_tool_error=False
        ),
        WebPage(
            name='WebPage',
            description='Worker that can get web pages through url. Useful when you have a  url and need to find 
detailed information.You must make sure that the url is real and correct, come from plugin or user input.Input 
should be a url.',
            args_schema=<class 'pydantic.main.WebPageArgs'>,
            verbose=False,
            handle_tool_error=False
        ),
        OpenAIFunctionChatAgent(
            name='main',
            type=<AgentType.openai: 'openai'>,
            version='0.0.1',
            description='main agent leveraging OpenAI function call API.',
            target_tasks=['anything'],
            llm=OpenAIGPTClient(
                model_name='gpt-4-0613',
                params=OpenAIParamModel(
                    max_tokens=1024,
                    temperature=0.0,
                    top_p=0.9,
                    presence_penalty=0.0,
                    frequency_penalty=0.0,
                    n=1,
                    stop=[]
                )
            ),
            prompt_template=PromptTemplate(
                input_variables=['instruction'],
                output_parser=None,
                partial_variables={},
                template='{instruction}',
                template_format='f-string',
                validate_template=True
            ),
            plugins=[
                GoogleSearch(
                    name='GoogleSearch',
                    description='Tool that searches results from Google. Input should be a search query.',
                    args_schema=<class 'pydantic.main.GoogleSearchArgs'>,
                    verbose=False,
                    handle_tool_error=False
                ),
                WebPage(
                    name='WebPage',
                    description='Worker that can get web pages through url. Useful when you have a  url and need to
find detailed information.You must make sure that the url is real and correct, come from plugin or user input.Input
should be a url.',
                    args_schema=<class 'pydantic.main.WebPageArgs'>,
                    verbose=False,
                    handle_tool_error=False
                )
            ],
            args_schema=<class 'pydantic.main.ArgsSchema'>,
            memory=None,
            examples=None,
            message_scratchpad=[{'role': 'system', 'content': 'You are a helpful AI assistant.'}]
        )
    ],
    args_schema=<class 'pydantic.main.ArgsSchema'>,
    memory=None,
    examples=None,
    message_scratchpad=[{'role': 'system', 'content': 'You are a helpful AI assistant.'}]
)

This is a sample agent that uses the `!env` syntax to reference environment variables.

In [6]:
import os
os.environ['AGENT_NAME'] = 'gentopia'

In [7]:
assembler = AgentAssembler(file='configs/env.yaml')
agent = assembler.get_agent()
print(agent)

OpenAIFunctionChatAgent(
    name='gentopia',
    type=<AgentType.openai: 'openai'>,
    version='0.0.1',
    description='main agent leveraging OpenAI function call API.',
    target_tasks=['anything'],
    llm=OpenAIGPTClient(
        model_name='gpt-4-0613',
        params=OpenAIParamModel(
            max_tokens=1024,
            temperature=0.0,
            top_p=0.9,
            presence_penalty=0.0,
            frequency_penalty=0.0,
            n=1,
            stop=[]
        )
    ),
    prompt_template=PromptTemplate(
        input_variables=['instruction'],
        output_parser=None,
        partial_variables={},
        template='{instruction}',
        template_format='f-string',
        validate_template=True
    ),
    plugins=[
        GoogleSearch(
            name='GoogleSearch',
            description='Tool that searches results from Google. Input should be a search query.',
            args_schema=<class 'pydantic.main.GoogleSearchArgs'>,
            verbose=False,
            handle_tool_error=False
        ),
        WebPage(
            name='WebPage',
            description='Worker that can get web pages through url. Useful when you have a  url and need to find 
detailed information.You must make sure that the url is real and correct, come from plugin or user input.Input 
should be a url.',
            args_schema=<class 'pydantic.main.WebPageArgs'>,
            verbose=False,
            handle_tool_error=False
        ),
        OpenAIFunctionChatAgent(
            name='main',
            type=<AgentType.openai: 'openai'>,
            version='0.0.1',
            description='main agent leveraging OpenAI function call API.',
            target_tasks=['anything'],
            llm=OpenAIGPTClient(
                model_name='gpt-4-0613',
                params=OpenAIParamModel(
                    max_tokens=1024,
                    temperature=0.0,
                    top_p=0.9,
                    presence_penalty=0.0,
                    frequency_penalty=0.0,
                    n=1,
                    stop=[]
                )
            ),
            prompt_template=PromptTemplate(
                input_variables=['instruction'],
                output_parser=None,
                partial_variables={},
                template='{instruction}',
                template_format='f-string',
                validate_template=True
            ),
            plugins=[
                GoogleSearch(
                    name='GoogleSearch',
                    description='Tool that searches results from Google. Input should be a search query.',
                    args_schema=<class 'pydantic.main.GoogleSearchArgs'>,
                    verbose=False,
                    handle_tool_error=False
                ),
                WebPage(
                    name='WebPage',
                    description='Worker that can get web pages through url. Useful when you have a  url and need to
find detailed information.You must make sure that the url is real and correct, come from plugin or user input.Input
should be a url.',
                    args_schema=<class 'pydantic.main.WebPageArgs'>,
                    verbose=False,
                    handle_tool_error=False
                )
            ],
            args_schema=<class 'pydantic.main.ArgsSchema'>,
            memory=None,
            examples=None,
            message_scratchpad=[{'role': 'system', 'content': 'You are a helpful AI assistant.'}]
        )
    ],
    args_schema=<class 'pydantic.main.ArgsSchema'>,
    memory=None,
    examples=None,
    message_scratchpad=[{'role': 'system', 'content': 'You are a helpful AI assistant.'}]
)